In [40]:
import torch
import os
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch import nn, optim
import pandas as pd
import numpy as np
from PIL import Image
from time import time

In [41]:
class TrainDatasetFromImages(Dataset):
    def __init__(self, csv_path):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        self.to_tensor = transforms.ToTensor()
        # Read the csv file
        self.data_info = pd.read_csv(csv_path, header=None)
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open("data/training/"+single_image_name)

        t = transforms.Resize((20,57))
        
        img_as_img = t(img_as_img)
        
        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]
        
        single_image_label = torch.tensor([single_image_label], dtype=torch.long)
        
        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len

#if __name__ == "__main__":
    # Call dataset
#    custom_mnist_from_images =  \
#        CustomDatasetFromImages('data/training')

In [42]:

train_data = TrainDatasetFromImages('data/training/labels-tab-csv.csv')




In [43]:
input_size = 1140
hidden_sizes = [128, 64]
output_size = 11 #might be an issue

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))

In [44]:
criterion = nn.NLLLoss()
images, labels = next(iter(train_data))
images = images.view(images.shape[0], -1)
logps = model(images) 
print(images.shape, labels.shape)
loss = criterion(logps, labels)


optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in train_data:
        images = images.view(images.shape[0], -1)
    
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_data)))

print("\nTraining Time (in minutes) =",(time()-time0)/60)

torch.Size([1, 1140]) torch.Size([1])
Epoch 0 - Training loss: 2.3456227640772975
Epoch 1 - Training loss: 2.323869098618973
Epoch 2 - Training loss: 2.319578746307728
Epoch 3 - Training loss: 2.3181707565174547
Epoch 4 - Training loss: 2.3157852813254953
Epoch 5 - Training loss: 2.3145249746566594
Epoch 6 - Training loss: 2.3136196951533474
Epoch 7 - Training loss: 2.3128278202788772
Epoch 8 - Training loss: 2.31207291608633
Epoch 9 - Training loss: 2.3112027112827747
Epoch 10 - Training loss: 2.3117198894190234
Epoch 11 - Training loss: 2.31128600170446
Epoch 12 - Training loss: 2.3099552007608635
Epoch 13 - Training loss: 2.309369156526965
Epoch 14 - Training loss: 2.3086506372274354

Training Time (in minutes) = 0.6564666787783305


In [45]:
valloader = TrainDatasetFromImages('data/training/labels-tab-csv.csv')

correct_count, all_count = 0, 0
for images,labels in valloader:
  for i in range(len(labels)):
    img = images[i].view(1, 1140)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    print(pred_label)
    
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))



6
6
6
6
6
6
6
6
6
6
5
5
6
5
6
6
1
6
6
6
1
5
6
6
6
6
6
6
6
6
5
6
6
6
6
6
1
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
5
5
6
5
6
6
6
6
6
6
5
6
6
5
6
6
6
1
5
6
6
6
6
1
6
6
6
5
5
6
5
6
6
1
6
6
6
1
1
6
6
6
6
1
6
6
6
5
5
6
5
6
6
1
6
6
5
6
6
6
6
6
6
5
6
6
6
1
1
5
6
6
6
1
6
6
6
1
5
5
6
6
6
1
6
5
6
6
6
6
6
6
6
5
6
6
6
6
6
6
5
6
6
6
6
6
6
5
6
6
6
6
6
5
6
6
6
1
6
6
6
6
6
6
6
6
6
1
1
5
1
7
8
8
6
6
6
8
6
5
6
6
6
5
6
8
6
8
8
8
8
8
6
8
10
8
10
5
6
6
6
6
6
6
6
6
6
1
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
1
5
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
1
6
6
6
6
6
6
6
6
6
5
6
5
6
6
6
6
6
6
6
5
6
6
6
5
5
6
6
6
6
1
6
6
6
5
6
6
6
6
6
1
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
5
6
6
1
6
6
6
8
8
5
5
8
6
8
8
10
10
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
5
6
5
6
6
6
6
6
6
6
6
6
6
6
5
6
6
5
5
5
1
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
1
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
1
6
6
6
5
6
1
5
6
6
1
6
6
6
1
3
3
6
6
6
1
6
6
6
6
6
6
6
6
6
1
6
6
6
6
3
5
5
5
5
5
5
